<a href="https://colab.research.google.com/github/DJCordhose/transformers/blob/main/notebooks/mistral-half-V100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



* https://huggingface.co/mistralai/Mistral-7B-Instruct-v0.2
* https://huggingface.co/mistralai/Mixtral-8x7B-Instruct-v0.1
* https://towardsdatascience.com/mixtral-8x7b-understanding-and-running-the-sparse-mixture-of-experts-0e3fc7fde818
* https://huggingface.co/chat/


In [1]:
!nvidia-smi

Sat Dec 16 12:22:34 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P0              26W / 300W |      0MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [2]:
!pip install -qU transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.3/8.3 MB 29.2 MB/s eta 0:00:00


In [3]:
# https://github.com/googlecolab/colabtools/issues/3409
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [4]:
!pip install accelerate bitsandbytes -qU

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 17.0 MB/s eta 0:00:00


In [5]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# model_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"
# needs just under 16GB when using half precision, 8GB with 8Bit
# might need more memory when generating responses
model_id = "mistralai/Mistral-7B-Instruct-v0.2"

tokenizer = AutoTokenizer.from_pretrained(model_id)

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

In [6]:
# model = AutoModelForCausalLM.from_pretrained(model_id)

In [7]:
# compute_dtype = getattr(torch, "float16")
# from transformers import BitsAndBytesConfig

# bnb4_config = BitsAndBytesConfig(
#         load_in_4bit=True,
#         bnb_4bit_quant_type="nf4",
#         bnb_4bit_compute_dtype=compute_dtype,
#         bnb_4bit_use_double_quant=True,
# )
# bnb8_config = BitsAndBytesConfig(
#         load_in_8bit=True,
#         bnb_8bit_quant_type="nf4",
#         bnb_8bit_compute_dtype=compute_dtype,
#         bnb_8bit_use_double_quant=True,
# )
# bnb_config = bnb4_config
# model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config)

In [8]:
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16).to(0)

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [9]:
model

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (norm): MistralRMSNorm()
  

In [17]:
%%time

messages = [
    {"role": "user", "content": "Hallo, was ist die beste deutsche Krankenversicherung? Bitte auf Deutsch antworten"},
]

device = "cuda"
encodeds = tokenizer.apply_chat_template(messages, return_tensors="pt")
model_inputs = encodeds.to(device)

generated_ids = model.generate(model_inputs, max_new_tokens=1000, do_sample=True)
decoded = tokenizer.batch_decode(generated_ids)
print(decoded[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] Hallo, was ist die beste deutsche Krankenversicherung? Bitte auf Deutsch antworten [/INST] Hallo! Ich habe kein persönliches Erfahrungswissenen mit deutschen Krankenversicherungen, aber ich kann Ihnen einige der bekanntesten deutsche Krankenversicherungen vorschlagen, die häufig als hervorragend bewertet werden:

1. Techniker Krankenkasse (TK)
2. Barmer GEK
3. AOK (Allgemeine Ortskrankenkasse)
4. DAK (Deutsche Angestellten-Krankenkasse)
5. BARMER GEITOOS

Um herauszufinden, dass die beste Krankenversicherung für Sie ist, empfehle ich, folgende Faktoren in Betracht zu ziehen: Präferenz Ihrer Arztpraxen und Spezialisten, das Angebot an Behandlungen, die jährliche Beitragsummen, die Auswahl an Zusatzleistungen und die Kundenservicequalität.

Um die genauen Details zu erfahren und die beste Krankenversicherung für Ihre Bedürfnisse zu identifizieren, empfehlen Sie es mir, den Angeboten der oben genannten Krankenversicherungen näher zuzukommen und die Details zu vergleichen. Sie k

In [18]:
!nvidia-smi

Sat Dec 16 12:28:11 2023       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla V100-SXM2-16GB           Off | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P0              45W / 300W |  15112MiB / 16384MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--